In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
%matplotlib inline
import os
import sys

print (sys.stdout.encoding)

UTF-8


In [2]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [10]:
td = date.today().isoformat()
print (td)
lls = lambda x: [x]

dateiso = '-'.join(map(lambda x: str(int(x)), date.today().isoformat().split("-")))
dateiso

2021-03-02


'2021-3-2'

In [4]:
URL = 'http://fubon-ebrokerdj.fbs.com.tw/Z/ZC/ZCX/ZCX_2330.djhtm'

resp = requests.get(URL)
soup = BeautifulSoup(resp.text, 'html.parser')
table = soup.find_all('table', {'class': 't01'})[1]
values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': 't3n1'})))
ohlcV = {
    'O': 0,
    'H': 1,
    'L': 2,
    'C': 3,
    'V': 11
}
for k,v in ohlcV.items():
    ohlcV[k] = lls(values[ohlcV[k]])
    
ohlcV = pd.DataFrame(ohlcV)
ohlcV

,O,H,L,C,V
0,621.00,622.00,609.00,609.00,61884


In [5]:
URL= 'http://fubon-ebrokerdj.fbs.com.tw/z/zc/zcx/zcxD5.djjs?A=2330'

resp = requests.get(URL)
soup = BeautifulSoup(resp.text, 'html.parser')
table = soup.find_all('table', {'class': 't01'})[0]
values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': 't3n1'})))

MarginPS = {
    'MarginPurchaseBuy': 0,
    'MarginPurchaseCashRepayment' : 2,
    'MarginPurchaseLimit': 4,
    'MarginPurchaseSell': 1,
    'MarginPurchaseTodayBalance': 3,
    'MarginPurchaseYesterdayBalance': 7
}
for k,v in MarginPS.items():
    MarginPS[k] = lls(values[MarginPS[k]])
    
MarginPS.update({
    'date': [td],
    'stock_id': ['2330']
})
data = pd.DataFrame(MarginPS)
data



,MarginPurchaseBuy,MarginPurchaseCashRepayment,MarginPurchaseLimit,MarginPurchaseSell,MarginPurchaseTodayBalance,MarginPurchaseYesterdayBalance,date,stock_id
0,1473,102,6482595,1748,34840,55,2021-03-02,2330


In [11]:
url = "https://api.finmindtrade.com/api/v4/data"
parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "data_id": "2330",
    "start_date": "2021-02-26",
    "token": "", # 參考登入，獲取金鑰
}
data = requests.get(url, params=parameter)
data = data.json()
data = pd.DataFrame(data['data'])
#data
data.head()

,date,stock_id,MarginPurchaseBuy,MarginPurchaseCashRepayment,MarginPurchaseLimit,MarginPurchaseSell,MarginPurchaseTodayBalance,MarginPurchaseYesterdayBalance,Note,OffsetLoanAndShort,ShortSaleBuy,ShortSaleCashRepayment,ShortSaleLimit,ShortSaleSell,ShortSaleTodayBalance,ShortSaleYesterdayBalance
0,2021-02-26,2330,4383,173,6482595,1119,35217,32126,,16,164,0,6482595,96,690,758
1,2021-03-02,2330,1473,102,6482595,1748,34840,35217,,12,55,4,6482595,76,707,690


In [14]:
url = "https://api.finmindtrade.com/api/v4/data"
parameter = {
    "dataset": "TaiwanStockSecuritiesLending",
    "data_id": "8044",
    "start_date": "2021-02-01",
    "token": "", # 參考登入，獲取金鑰
}
data = requests.get(url, params=parameter)
data = data.json()
data = pd.DataFrame(data['data'])
print(data.head())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# 使用insert 0即只使用github，避免交叉使用了pip安装的abupy，导致的版本不一致问题
sys.path.insert(0, os.path.abspath('../'))
import abupy

In [ ]:
# 使用沙盒数据，目的是和书中一样的数据环境
abupy.env.disable_example_env_ipython()

In [ ]:
abupy.env.g_is_ipython = 0

In [ ]:
abupy.env.g_data_fetch_mode

In [ ]:
abupy.env.g_data_fetch_mode = abupy.EMarketDataFetchMode.E_DATA_FETCH_FORCE_NET

In [ ]:
abupy.ABuSymbolPd.make_kl_df('2330', n_folds=4).tail()

In [ ]:
from abupy import AbuDoubleMaBuy, AbuDoubleMaSell, AbuSDBreak, AbuFactorBuyBreak
from abupy import AbuFactorCloseAtrNStop, AbuFactorAtrNStop, AbuFactorPreAtrNStop
from abupy import abu, ABuProgress, AbuMetricsBase, EMarketTargetType, ABuMarketDrawing
from abupy import AbuFactorSellNDay, AbuFactorBuyWD, AbuFactorSellBreak
from abupy import AbuSymbolTW

In [ ]:
buy_factors = [{'class': AbuDoubleMaBuy},
               {'xd': 21, 'class': AbuFactorBuyBreak}, 
               {'xd': 7, 'class': AbuSDBreak}]

sell_factors = [
    {'class': AbuDoubleMaSell},
    {'xd': 21, 'class': AbuFactorSellBreak},
    {'stop_loss_n': 1.0, 'stop_win_n': 3.0, 'class': AbuFactorAtrNStop},
    {'class': AbuFactorPreAtrNStop, 'pre_atr_n': 1.5},
    {'class': AbuFactorCloseAtrNStop, 'close_atr_n': 1.5}
]

In [ ]:
# 使用沙盒内的美股做为回测目标
choice_symbols = ['2330']

# 初始资金量
read_cash = 3000000
# 使用run_loop_back运行策略
abu_result_tuple, kl_pd_manger = abu.run_loop_back(read_cash,
                                                   buy_factors,
                                                   sell_factors,
                                                   n_folds=6,
                                                   choice_symbols=choice_symbols)
ABuProgress.clear_output()

In [ ]:
AbuMetricsBase.show_general(*abu_result_tuple, only_show_returns=True)

In [ ]:
orders_pd = abu_result_tuple.orders_pd
view_orders = orders_pd[(orders_pd['symbol'] == '2330')]
view_orders

In [ ]:
trade_df = ABuMarketDrawing.plot_candle_from_order(view_orders)